In [1]:
using Revise

In [2]:
using RigidBodyDynamics
using LearningMPC
using Gurobi
using DrakeVisualizer
using CoordinateTransformations
using Plots; gr()
using ProgressMeter
using MLDataPattern
using JLD2
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [3]:
import BoxValkyries
import Nets

INFO: Precompiling module BoxValkyries.


In [4]:
reload("BoxValkyries")

In [5]:
const Sample = Tuple{Vector{Float64}, Matrix{Float64}}

Tuple{Array{Float64,1},Array{Float64,2}}

In [17]:
boxval = BoxValkyries.BoxValkyrie()
mechanism = boxval.mechanism
basevis = Visualizer()[:boxval]
delete!(basevis)
setgeometry!(basevis, boxval)

train_data = Sample[]
test_data = Sample[]
validation_data = Sample[]

x_to_u = AffineMap(eye(4), zeros(4))
v_to_y = AffineMap([max(abs(b.lower), abs(b.upper)) for b in LCPSim.all_effort_bounds(mechanism)], zeros(2))

widths = [4, 16, 16, 8, 2]
activation = Nets.leaky_relu
function sensitive_loss(λ)
    q = [1.0-λ λ λ λ λ]
    (params, x, y) -> sum(abs2, 
        q .* (Nets.predict_sensitivity(Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y), x) .- y))
end
start_params = 0.1 * randn(Nets.Params{Float64}, widths).data;

p_train = 0.6
p_validate = 0.2

0.2

In [61]:
function nominal_input(x0::MechanismState)
    u_nominal = clamp.(inverse_dynamics(x0, zeros(num_velocities(x0))), LCPSim.all_effort_bounds(x0.mechanism))
    feet = findbody.(x0.mechanism, ["rf", "lf"])
    weight = mass(x0.mechanism) * mechanism.gravitational_acceleration.v[3]
    u_nominal[parentindexes(velocity(x0, findjoint(x0.mechanism, "core_to_rf_z")))...] += weight / 2
    u_nominal[parentindexes(velocity(x0, findjoint(x0.mechanism, "core_to_lf_z")))...] += weight / 2
    u_nominal
end

nominal_input (generic function with 1 method)

In [115]:
mpc_params = BoxValkyries.BoxValkyrieMPCParams(Δt = 0.04, gap=1e-2)
xstar = MechanismState{Float64}(mechanism)
set_configuration!(xstar, findjoint(mechanism, "base_x"), [0])
set_configuration!(xstar, findjoint(mechanism, "base_z"), [0.9])
set_configuration!(xstar, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(xstar, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(xstar, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(xstar, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(xstar, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(xstar, findjoint(mechanism, "core_to_rf_z"), [-0.9])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_z"), [-0.9])
settransform!(basevis[:robot], xstar)

feet = findbody.(mechanism, ["rf", "lf"])
contacts = [Point3D(default_frame(body), 0., 0, 0) for body in feet]
lqrsol = BoxValkyries.LQRSolution(xstar, nominal_input(xstar), mpc_params.Q, mpc_params.R, contacts)
lqr_controller = BoxValkyries.LQRController(lqrsol)

(::#5) (generic function with 1 method)

In [116]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [-0.2])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1.2])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0));

In [117]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
env = Gurobi.Env()
results_lqr = LCPSim.simulate(x0, lqr_controller, boxval.environment, mpc_params.Δt, 100, GurobiSolver(env, OutputFlag=0));

In [ ]:
playback(basevis[:robot], results_lqr, mpc_params.Δt)

In [ ]:
jldopen("box-val-dagger-0.04-params.jld2", "r") do file
    params = file["params"]
    widths = file["widths"]
    x_to_u = file["x_to_u"]
    v_to_y = file["v_to_y"]
end;

In [ ]:
# params = copy(start_params)
# net = Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y)

In [ ]:
net_controller = x -> begin
    Nets.predict(net, state_vector(x))
end

mpc_controller = CartPoles.MPCController(cartpole, cartpole_mpc_params, xstar, lqrsol, [net_controller, lqr_controller]);
mpc_controller.callback = (x, results) -> begin
    if !isnull(results.lcp_updates)
#         playback(basevis[:robot], get(results.lcp_updates), mpc_controller.params.Δt)
        if !isnull(results.jacobian)
            xv = state_vector(x)
            yJ = hcat(get(results.lcp_updates)[1].input, get(results.jacobian))
            r = rand()
            if r < p_train
                push!(train_data, (xv, yJ))
            elseif r < p_train + p_validate
                push!(validation_data, (xv, yJ))
            else
                push!(test_data, (xv, yJ))
            end
        end
    end
end

train_loss = sensitive_loss(0.1)
validate_loss = train_loss
training_losses = Float64[]
validation_losses = Float64[]

p_mpc = 0.2

x_control = MechanismState{Float64}(mechanism)

controller = x -> begin
    set_configuration!(x_control, configuration(x))
    settransform!(basevis[:robot], x_control)
    if rand() < p_mpc
        return mpc_controller(x)
    else
        return net_controller(x)
    end
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)

fname = "cart-pole-dagger-0.04.jld2"

@showprogress for i in 1:1000
    set_configuration!(x0, [2 * (rand() - 0.5), π * (rand() - 0.5)])
    set_velocity!(x0, (rand(2) .- 0.5))
    termination = x -> (configuration(x)[2] > π/2 || configuration(x)[2] < -π/2)
    results_net = LCPSim.simulate(x0, controller, cartpole.environment, cartpole_mpc_params.Δt, 100, GurobiSolver(OutputFlag=0);
        termination=termination)
    Nets.adam!(train_loss, params, shuffleobs(train_data), Nets.AdamOpts(learning_rate=0.01 * 0.999^i, batch_size=min(10, length(train_data))))
    push!(training_losses, mean(xy -> validate_loss(params, xy[1], xy[2]), train_data))
    push!(validation_losses, mean(xy -> validate_loss(params, xy[1], xy[2]), validation_data))
    
    @show training_losses[end]
    @show validation_losses[end]
    
    jldopen(fname, "w") do file
        file["train_data"] = train_data
        file["test_data"] = test_data
        file["validation_data"] = validation_data
    end
    
    jldopen("cart-pole-dagger-0.04-params.jld2", "w") do file
        file["params"] = params
        file["widths"] = widths
        file["x_to_u"] = x_to_u
        file["v_to_y"] = v_to_y
        file["training_losses"] = training_losses
        file["validation_losses"] = validation_losses
    end;
end 

In [ ]:
q0 = [2 * (rand() - 0.5), π * (rand() - 0.5)]
v0 = (rand(2) .- 0.5);

In [ ]:
configuration(x0) .= q0
velocity(x0) .= v0
results_net = LCPSim.simulate(x0, net_controller, cartpole.environment, cartpole_mpc_params.Δt, 300, GurobiSolver(OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results_net[1].state))
settransform!(basevis[:robot], x0)

playback(basevis[:robot], results_net, 0.5 * cartpole_mpc_params.Δt)

In [ ]:
configuration(x0) .= q0
velocity(x0) .= v0
results_net = LCPSim.simulate(x0, lqr_controller, cartpole.environment, cartpole_mpc_params.Δt, 300, GurobiSolver(OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results_net[1].state))
settransform!(basevis[:robot], x0)

playback(basevis[:robot], results_net, 0.5 * cartpole_mpc_params.Δt)

In [ ]:
online_controller = CartPoles.OnlineMPCController(cartpole, 
    cartpole_mpc_params, xstar, lqrsol, [net_controller, lqr_controller]);

In [ ]:
configuration(x0) .= q0
velocity(x0) .= v0
results_net = LCPSim.simulate(x0, online_controller, cartpole.environment, cartpole_mpc_params.Δt, 300, GurobiSolver(OutputFlag=0));

In [ ]:
using ProfileView

Profile.clear()
@time @profile online_controller(x0)
ProfileView.view()